In [ ]:
#Imports

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
from scipy.spatial import distance
import os

In [ ]:
#loading haarcascade_frontalface_default.xml
face_model = cv2.CascadeClassifier('../input/haarcascades/haarcascade_frontalface_default.xml')

In [ ]:
import matplotlib.pyplot as plt
#trying it out on a sample image
img = cv2.imread('../input/face-mask-detection/images/maksssksksss244.png')

img = cv2.cvtColor(img, cv2.IMREAD_GRAYSCALE)

faces = face_model.detectMultiScale(img,scaleFactor=1.1, minNeighbors=4) #returns a list of (x,y,w,h) tuples

out_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) #colored output image

#plotting
for (x,y,w,h) in faces:
    cv2.rectangle(out_img,(x,y),(x+w,y+h),(0,0,255),1)
plt.figure(figsize=(12,12))
plt.imshow(out_img)

In [ ]:
#Imports for model training

from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras import Sequential
from keras.layers import Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
#Line necessary for distributing training over multiple 2 GPUs (NVIDIA T4 x2)

mirrored_strategy = tf.distribute.MirroredStrategy()

In [ ]:
#Define labels and our dataframe

data = pd.DataFrame(columns=['image_path', 'label'])
labels = {
    '/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithMask' : 'WithMask',
    '/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Train/WithMask' : 'WithMask',
    '/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Validation/WithMask' : 'WithMask',
    '/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithoutMask' : 'NoMask',
    '/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Train/WithoutMask' : 'NoMask',
    '/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Validation/WithoutMask' : 'NoMask'
}

In [ ]:
#Combining the training, test and validation sets into one dataframe

for folder in labels:
    for image_name in os.listdir(folder):
        image_path = os.path.join(folder, image_name)
        label = labels[folder]
        data = pd.concat([data, pd.DataFrame({'image_path': [image_path], 'label': [label]})], ignore_index=True)

In [ ]:
#Save the dataframe containing the whole labeled dataset as a .csv file in the working directory

data.to_csv('dataset.csv')

In [ ]:
#Reading the csv and converting it back to a dataframe

df = pd.read_csv("/kaggle/working/dataset.csv")
print(df)

In [ ]:
# Split the whole dataset into training and testing sets

train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

In [ ]:
# Data augmentation
train_datagen = ImageDataGenerator(rescale=1.0/255, horizontal_flip=True, zoom_range=0.2, shear_range=0.2)
train_generator = train_datagen.flow_from_dataframe(dataframe=train_df, x_col='image_path', y_col='label',
                                                    target_size=(224, 224), class_mode='categorical', batch_size=32)

test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_dataframe(dataframe=test_df, x_col='image_path', y_col='label',
                                                  target_size=(224, 224), class_mode='categorical', batch_size=32)


In [ ]:
!pip install deepface

In [ ]:
from deepface import DeepFace

In [ ]:
#Defining function for model creation using VGG19 as our base model

def create_model():
    # Load the DeepFace VGGFace model with the specified input shape
    base_model = DeepFace.build_model('VGG-Face')

    # Freeze the early layers of the VGGFace model
    for layer in base_model.layers[:-3]:
        layer.trainable = False

    # Create a new model with the VGGFace base and additional layers
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(2, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
# Function to extract SIFT features from an image
def extract_sift_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return descriptors

In [ ]:
# Extract SIFT features for each image in the training set
# train_sift_features = []
# train_labels = []

for index, row in train_df.iterrows():
    image_path = row['image_path']
    label = row['label']
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
#     sift_features = extract_sift_features(img)
    
#     # Check if features are extracted
#     if sift_features is not None:
#         # If multiple features are extracted, take the mean
#         if len(sift_features) > 1:
#             sift_features = np.mean(sift_features, axis=0, keepdims=True)
#         train_sift_features.append(sift_features)
#     else:
#         # If no features are extracted, append a zero-filled array
#         train_sift_features.append(np.zeros((1, 128), dtype=np.float32))  # Assuming 128 is the size of SIFT descriptors
    train_labels.append(label)
        
    # Print the number of features extracted for the current image
#     print(f"Image: {image_path}, Number of SIFT features: {sift_features.shape[0] if sift_features is not None else 0}")

# Convert lists to numpy arrays
# train_sift_features = np.vstack(train_sift_features)

# Verify the shapes
# print("Train SIFT Features shape:", train_sift_features.shape)
# print("Train Labels shape:", len(train_labels))

# Extract SIFT features for each image in the testing set
# test_sift_features = []
# test_labels = []

for index, row in test_df.iterrows():
    image_path = row['image_path']
    label = row['label']
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
#     sift_features = extract_sift_features(img)
    
    # Check if features are extracted
#     if sift_features is not None:
#         # If multiple features are extracted, take the mean
#         if len(sift_features) > 1:
#             sift_features = np.mean(sift_features, axis=0, keepdims=True)
#         test_sift_features.append(sift_features)
#     else:
#         # If no features are extracted, append a zero-filled array
#         test_sift_features.append(np.zeros((1, 128), dtype=np.float32))  # Assuming 128 is the size of SIFT descriptors
    test_labels.append(label)

    # Print the number of features extracted for the current image
#     print(f"Image: {image_path}, Number of SIFT features: {sift_features.shape[0] if sift_features is not None else 0}")

# Convert lists to numpy arrays
# test_sift_features = np.vstack(test_sift_features)

# Verify the shapes
# print("Test SIFT Features shape:", test_sift_features.shape)
# print("Test Labels shape:", len(test_labels))

In [ ]:
from keras.utils import to_categorical

# Convert labels to numeric categorical labels
label_mapping = {'WithMask': 0, 'NoMask': 1}  # Define your label mapping
train_labels_numeric = np.array([label_mapping[label] for label in train_labels])
test_labels_numeric = np.array([label_mapping[label] for label in test_labels])

# One-hot encode the numeric labels
train_labels_onehot = to_categorical(train_labels_numeric)
test_labels_onehot = to_categorical(test_labels_numeric)

In [ ]:
# print("Train data shapes:", train_sift_features.shape, train_labels_onehot.shape)
# print("Test data shapes:", test_sift_features.shape, test_labels_onehot.shape)

In [ ]:
# num_sift_features = 128  # Change this to the actual size of your SIFT descriptors
# target_height, target_width, num_channels = 224, 224, 3

# # Reshape the SIFT features
# train_sift_features_reshaped = train_sift_features.reshape(-1, num_sift_features, 1, 1)
# test_sift_features_reshaped = test_sift_features.reshape(-1, num_sift_features, 1, 1)

# # Repeat the SIFT features along the channel axis to match the required number of channels (3)
# train_sift_features_reshaped = np.repeat(train_sift_features_reshaped, num_channels, axis=2)
# test_sift_features_reshaped = np.repeat(test_sift_features_reshaped, num_channels, axis=2)

# # Resize the reshaped SIFT features to match the target image size
# train_sift_features_reshaped = np.repeat(train_sift_features_reshaped, target_height, axis=3)
# test_sift_features_reshaped = np.repeat(test_sift_features_reshaped, target_height, axis=3)

In [ ]:
with mirrored_strategy.scope():
        model = create_model()
        history = model.fit(train_generator, epochs=10, batch_size=32, validation_data=test_generator)

In [ ]:
# Evaluate the model on the test set for this fold
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy for Fold {fold}: {test_accuracy}")

In [ ]:
# @title Loss & Accuracy Visualizations
import matplotlib.pyplot as plt

# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# Plot the loss
ax1.plot(history.history['loss'], label='Training Loss')
ax1.plot(history.history['val_loss'], label='Validation Loss')
ax1.legend()
ax1.set_title('Training and Validation Loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')

# Plot the accuracy
ax2.plot(history.history['accuracy'], label='Training Accuracy')
ax2.plot(history.history['val_accuracy'], label='Validation Accuracy')
ax2.legend()
ax2.set_title('Training and Validation Accuracy')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')

# Adjust layout and show the plots
plt.tight_layout()

# Save the figures in the working directory
plt.savefig('LossVal_loss.png')
plt.savefig('AccVal_acc.png')

# Show the plots
plt.show()